In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import norm
from statsmodels.base.model import GenericLikelihoodModel
import seaborn as sns
sns.set_palette("muted")
sns.set_color_codes()
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
sns.set_style({"axes.grid": "True", "grid.color": "0.95"})

plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams["figure.dpi"] = 100

In [ ]:
import matplotlib as mpl
def darken_color(color, p):
    return (color[0]*p,color[1]*p,color[2]*p)

colors = sns.color_palette("muted") + [(.1, .1, .1)]
for code, color in zip(["bd","gd","rd","md","yd","cd","kd"], colors):
    rgb = mpl.colors.colorConverter.to_rgb(darken_color(color,0.8))
    mpl.colors.colorConverter.colors[code] = rgb
    mpl.colors.colorConverter.cache[code] = rgb

Prepare variable list for reading the result files. `time_dependent` needs to be changed to `False` for time-independent results.

In [ ]:
from collections import OrderedDict 

In [ ]:
time_dependent = True

var_types = ("gen", "fit", "err")
var_dict_ti = OrderedDict([
            ("ap", r"$|A_\parallel|$"), 
            ("apa", r"$\arg(A_\parallel)$"), 
            ("a0", r"$|A_0|$"), 
            ("a0a", r"$\arg(A_0)$"), 
            ("at", r"$|A_\perp|$"), 
            ("ata", r"$\arg(A_\perp)$")
            ])

var_dict_td = OrderedDict([
            ("ap", r"$|A_\parallel|$"), 
            ("apa", r"$\arg(A_\parallel)$"), 
            ("a0", r"$|A_0|$"), 
            ("a0a", r"$\arg(A_0)$"), 
            ("at", r"$|A_\perp|$"), 
            ("ata", r"$\arg(A_\perp)$"),
            ("xp", r"$x_\parallel$"),
            ("x0", r"$x_0$"),
            ("xt", r"$x_\perp$"),
            ("yp", r"$y_\parallel$"),
            ("y0", r"$y_0$"),
            ("yt", r"$y_\perp$"),
            ("xbp", r"$\bar x_\parallel$"),
            ("xb0", r"$\bar x_0$"),
            ("xbt", r"$\bar x_\perp$"),
            ("ybp", r"$\bar y_\parallel$"),
            ("yb0", r"$\bar y_0$"),
            ("ybt", r"$\bar y_\perp$")
            ])

var_dict = {}
if time_dependent:
    var_dict = var_dict_td
else:
    var_dict = var_dict_ti

var_names = list(var_dict.keys())
vars = ([var_name + "_" + var_type for var_name in var_names for var_type in var_types])

In [ ]:
import os
import glob

dirs_ti = [
    '../results/Kpi_ti_CR',
    '../results/Kpipi0_ti_CR',
    '../results/K3pi_ti_CR',
    '../results/together_ti_CR',
    '../results/Kpi_ti_CRSCF',
    '../results/Kpipi0_ti_CRSCF',
    '../results/K3pi_ti_CRSCF',
    '../results/together_ti_CRSCF',
    '../results/Kpi_ti_all',
    '../results/Kpipi0_ti_all',
    '../results/K3pi_ti_all',
    '../results/together_ti_all'
]

dirs_td = [
    '../results/Kpi_td_CR',
    '../results/Kpipi0_td_CR',
    '../results/K3pi_td_CR',
    '../results/together_td_CR',
    '../results/Kpi_td_CRSCF',
    '../results/Kpipi0_td_CRSCF',
    '../results/K3pi_td_CRSCF',
    '../results/together_td_CRSCF',
    '../results/Kpi_td_all',
    '../results/Kpipi0_td_all',
    '../results/K3pi_td_all',
    '../results/together_td_all'
]

dirs = []
if (time_dependent):
    dirs = dirs_td
else:
    dirs = dirs_ti 

dfs = []
for directory in dirs:
    all_files = glob.glob(os.path.join(directory, "stream*[0-9]"))
    print("Num files in '" + str(directory) + "': " + str(len(all_files)))
    df_from_each_file = (pd.read_csv(f, sep=" \|\| | \| | ", header=None, names=vars, engine='python') for f in all_files)
    df = pd.concat(df_from_each_file, ignore_index=True)
    dfs.append(df)

Calculate the pulls and display their means and then standard deviations.

In [ ]:
dfs_pulls = []
for i in range(0, len(dirs)):
    df = dfs[i]
    df_pulls_dict = {var : (df[var + "_fit"] - df[var + "_gen"])/df[var + "_err"] for var in var_names}
    df_pulls = pd.DataFrame(df_pulls_dict)
    dfs_pulls.append(df_pulls)

In [ ]:
for var in var_names:
    print("{:4}| ".format(var), end='')
    for i in range(0, len(dirs)):
        print("{:+5.2f} | ".format(dfs_pulls[i].mean()[var]), end='')
    print()

In [ ]:
for var in var_names:
    print("{:4}| ".format(var), end='')
    for i in range(0, len(dirs)):
        print("{:+5.2f} | ".format(dfs_pulls[i].std()[var]), end='')
    print()

The following cell shows a distribution of pulls for all directories and all variables

In [ ]:
rows = 0
if time_dependent:
    rows = 6
else:
    rows = 2

plt.rcParams["figure.figsize"] = [9, rows * 3]
for i, dir in enumerate(dirs):
    axs = dfs_pulls[i].hist(column=list(var_names),
                            sharey=True, layout=(rows, 3), range=(-5, 5), bins=10)
    print("Plots for dir " + os.path.basename(dir))
    for ax in axs.flat:
        ax.set_xlabel(var_dict[ax.title.get_text()])
        if ax.title.get_text() == "a0a":
            ax.text(0.5, 0.5, "Empty on purpose", horizontalalignment="center", 
                    verticalalignment="center", transform=ax.transAxes)
        ax.set_title("")
        
    plt.savefig(os.path.basename(dir) + "_pull_dist.pdf", bbox_inches = 'tight')
    plt.show()

In [ ]:
class Gaussian(GenericLikelihoodModel):
    def __init__(self, endog, exog=None, **kwds):
        #if exog is None:
        #    exog = np.zeros_like(endog)
            
        super(Gaussian, self).__init__(endog, exog, **kwds)
    
    def nloglikeobs(self, params):
        loc = params[0]
        scale = params[1]

        return -np.log(norm.pdf(self.endog, loc=loc, scale=scale))
    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        if start_params is None:
            loc_start = self.endog.mean()
            scale_start = self.endog.std()
            
            start_params = np.array([loc_start, scale_start])
            
        return super(Gaussian, self).fit(start_params=start_params,
                                         maxiter=maxiter, maxfun=maxfun, **kwds)

This fits the pull distributions with Gaussians and shows the uncertainties on the $\mu$ and $\sigma$ of each distribution. 

In [ ]:
real_vars = list(var_names)
real_vars.remove('a0a')

for i, dir in enumerate(dirs):
    print("Results for dir " + dir)
    for var in real_vars:
        model = Gaussian(dfs_pulls[i][var]);
        results = model.fit(disp=False);
        print("{:4}: ({:+.2f} +- {:.2f}) +- ({:+.2f} +- {:.2f})".format(
            var, results.params[0], results.bse[0], results.params[1], results.bse[1]))
    print()

We now calculate the relative errors $|\sigma/\mu|$ for each variable and dataset

In [ ]:
# Calculate relative errors
dfs_relative_errors = []
for i in range(0, len(dirs)):
    df = dfs[i]
    df_relative_errors_dict = {var : abs(df[var + "_err"]/df[var + "_fit"]) for var in var_names}
    df_relative_errors = pd.DataFrame(df_relative_errors_dict)
    dfs_relative_errors.append(df_relative_errors)
#df_pulls = df_pulls.drop(list(var_names[0:6]), axis=1)

# Display relative errors in a table
for var in var_names:
    print("{:4}| ".format(var), end='')
    for i, dir in enumerate(dirs):
        print("{:6.2f} | ".format(dfs_relative_errors[i].mean()[var]), end='')
    print()

This might be useful if we want to plot the fitted Gaussian on top of the histograms.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("$\\tau$ [ps]")
#ax.set_xlim(1.5, 1.57)

data = dfs_pulls[0].apa

# Create histogram and calculate area under it for
# renormalization of fitted PDF
n, bins, patches = plt.hist(data, bins=20, edgecolor=darken_color(sns.color_palette("muted")[0], 0.8))
area = np.sum(np.diff(bins)*n)

mu, sigma = norm.fit(data)

# Create a bunch of equidistant points to calculate the 
# function values at (many points to make it look smooth)
x = np.linspace(data.min(), data.max(), 100)
norm_fitted = norm.pdf(x, mu, sigma)*area
plt.plot(x, norm_fitted)
print("tau = {:.3f} +- {:.3f} ps".format(mu, sigma))